# Transformando Empréstimo em um TAD

Considere a resolução para múltiplos empréstimos.

In [1]:
#include <stdio.h>

typedef struct {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
} Emprestimo;

Emprestimo novoEmprestimo(float s, int n, float j) {
    Emprestimo umEmprestimo;
    umEmprestimo.s = s;
    umEmprestimo.n = n;
    umEmprestimo.j = j;
    umEmprestimo.corrente = 1;
    umEmprestimo.p = s;
    return umEmprestimo;
}

float proximaParcela(Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->p;
    if (umEmprestimo->corrente < umEmprestimo->n)
        umEmprestimo->p = umEmprestimo->p + (umEmprestimo->p * (umEmprestimo->j/100));
    else
        umEmprestimo->p = 0;
    (umEmprestimo->corrente)++;
    return retorno;
}

int main() {
    Emprestimo emprestimo1,
               emprestimo2;
    
    emprestimo1 = novoEmprestimo(200, 5, 1);
    emprestimo2 = novoEmprestimo(500, 7, 2);
    
    float p1 = proximaParcela(&emprestimo1),
          p2 = proximaParcela(&emprestimo2);
    
    int i = 1;
    while (p1 > 0 || p2 > 0) {
        if (p1 > 0)
            printf("Emprestimo 1: parcela %d eh %3.2f\n", i, p1);
        if (p2 > 0)
            printf("Emprestimo 2: parcela %d eh %3.2f\n", i, p2);
        p1 = proximaParcela(&emprestimo1);
        p2 = proximaParcela(&emprestimo2);
        i++;
    }   
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08


# Exercício

Modifique este código (ou um código que você mesmo tenha feito) para que a estrutura (struct) possa representar um Tipo Abstrato de Dados (TAD), em que ela contenha não apenas os valores de estado,como também operações executadas sobre eles. As operações (funções em C) também devem fazer parte do struct do mesmo modo que os campos de dados.

## Solução

Lidar com structs alocadas na pilha se provou um pouco trabalhoso, então para a implementação do TAD as funções foram reescritas para lidarem com structs alocadas na heap, de modo que:

1. Os objetos não sejam destruídos quando saírem de escopo
2. As funções consigam acessar diretamente os objetos por meio de ponteiros

Além disso, a implementação das operações como membros das structs se deu por meio da implementação de um membro `self` que contém um ponteiro para o próprio objeto. Os métodos do objeto recebem sempre como primeiro argumento o ponteiro `self` (de modo parecido com o que acontece em Python), e.g. 
```c
void metodo(Emprestimo* self, ...);
obj->metodo(obj->self, ...);
```

In [39]:
#include <stdio.h>
#include <stdlib.h>

typedef struct _Emprestimo {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
    
    struct _Emprestimo* self;
    float (*proximaParcela)(struct _Emprestimo* self);
} Emprestimo;

float proximaParcela(Emprestimo *self) {
    float retorno = self->p;
    if (self->corrente < self->n)
        self->p = self->p + (self->p * (self->j/100));
    else
        self->p = 0;
    (self->corrente)++;
    return retorno;
}

Emprestimo* novoEmprestimo(float s, int n, float j) {
    Emprestimo* umEmprestimo = malloc(sizeof(Emprestimo));
    umEmprestimo->s = s;
    umEmprestimo->n = n;
    umEmprestimo->j = j;
    umEmprestimo->corrente = 1;
    umEmprestimo->p = s;
    
    umEmprestimo->self = umEmprestimo;
    umEmprestimo->proximaParcela = &proximaParcela;
    
    return umEmprestimo;
}

int main() {
    Emprestimo *emprestimo1,
               *emprestimo2;
    
    emprestimo1 = novoEmprestimo(200, 5, 1);
    emprestimo2 = novoEmprestimo(500, 7, 2);
    
    float p1 = emprestimo1->proximaParcela(emprestimo1->self),
          p2 = emprestimo1->proximaParcela(emprestimo2->self);
    
    int i = 1;
    while (p1 > 0 || p2 > 0) {
        if (p1 > 0)
            printf("Emprestimo 1: parcela %d eh %3.2f\n", i, p1);
        if (p2 > 0)
            printf("Emprestimo 2: parcela %d eh %3.2f\n", i, p2);
        p1 = emprestimo1->proximaParcela(emprestimo1->self);
        p2 = emprestimo2->proximaParcela(emprestimo2->self);
        i++;
    }   
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
